In [ ]:
import pandas as pd
from collections import Counter
import ast

# Load datasets
surveyor_data_df = pd.read_csv('/content/drive/MyDrive/surveyor_data.csv')
grouped_part_df = pd.read_csv('/content/drive/MyDrive/grouped_part_descriptions3.csv')

# Step 1: Find Top 10 Most Commonly Damaged Primary Parts
primary_part_counts = surveyor_data_df['TXT_PARTS_NAME'].value_counts().head(10)
top_primary_parts = primary_part_counts.index.tolist()

# Convert PARTDESCRIPTION column from string to list
grouped_part_df['PARTDESCRIPTION'] = grouped_part_df['PARTDESCRIPTION'].apply(ast.literal_eval)

def find_similar_primary_parts(primary_parts, available_parts):
    """Find closest matching primary parts based on name similarity."""
    return {p: next((a for a in available_parts if p.lower() in a.lower()), None) for p in primary_parts}

# Match primary parts between datasets
corrected_primary_mapping = find_similar_primary_parts(top_primary_parts, set(grouped_part_df['TXT_PARTS_NAME'].unique()))
corrected_primary_mapping = {k: v for k, v in corrected_primary_mapping.items() if v}  # Remove unmatched

# Step 2: Find Top 10 Secondary Parts Associated with Each Primary Part
secondary_part_mapping = {primary: Counter() for primary in corrected_primary_mapping.keys()}

for _, row in grouped_part_df.iterrows():
    primary_part = row['TXT_PARTS_NAME']
    for key, corrected in corrected_primary_mapping.items():
        if corrected == primary_part:
            secondary_part_mapping[key].update(row['PARTDESCRIPTION'])

# Extract the top 10 secondary parts per primary part
top_secondary_parts_per_primary = {
    primary: sec_parts.most_common(10) for primary, sec_parts in secondary_part_mapping.items()
}

# Display Results
print("Top 10 Most Commonly Damaged Primary Parts:")
print(primary_part_counts)
print("\nTop 10 Secondary Parts Associated with Each Primary Part:")
for primary, secondaries in top_secondary_parts_per_primary.items():
    print(f"\n{primary}:")
    for part, count in secondaries:
        print(f"  - {part} ({count})")


Top 10 Most Commonly Damaged Primary Parts:
TXT_PARTS_NAME
Bumper Front Assembly               55074
Bumper Rear Assembly                23974
Head Light Left                     23447
Head Light Right                    21415
Windshield Glass Front              18355
Sealant Front Windshield Glass 1    17169
Fender Panel Front Left             16058
Fender Panel Front Right            15749
Bonnet|Hood Assembly                14952
Moulding Front Windshield           13535
Name: count, dtype: int64

Top 10 Secondary Parts Associated with Each Primary Part:

Bumper Front Assembly:
  - bumperfront (1178)
  - clip (1176)
  - bumper front (672)
  - bolt (515)
  - holderfr bumper sider (505)
  - panelfront fenderl (468)
  - nut (447)
  - panelfront fenderr (435)
  - holderfr bumper sidel (420)
  - condenser assy (390)

Bumper Rear Assembly:
  - clip (448)
  - bumperrear (312)
  - bumperfront (307)
  - holderrr bumper sider (210)
  - bumper front (187)
  - dg sealant set without primer (185